In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [6]:
df = pd.read_csv('memphis_restaurants.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [8]:
df_small = df[0:5]

In [5]:
for row in df_small.iterrows():
    try:
        #### Get the business name and url ####
        business_name = row[1]['name']
        business_url = row[1]['url']
        print(business_name, business_url)

        #### Request the page ####
        html = requests.get(business_url)
        soup = BeautifulSoup(html.content, 'html.parser')

        #### Get the number of pages for reviews and limit it to 10 ####
#         n_reviews = soup.select('.text-align--center__373c0__1l506 .css-e81eai')
#         try:
#             n_review_pg = int(re.sub('1 of ', '',n_reviews[0].string))
#             if n_review_pg > 10:
#                 n_review_pg = 10
#         except:
#             n_review_pg = 10

        #### Get the data for each of the review pages ####

        for i in range(5):
            try:
                url2 = business_url + '?start=' + str(i * 10)
                html = requests.get(url2)
                soup = BeautifulSoup(html.content, 'html.parser')

                #### Names
                names = soup.select('.css-m6anxm .css-166la90')
                review_names = []

                for name in names:
                    review_names.append(name.string)

                #### Ratings
                ratings = soup.select('.margin-b1-5__373c0__2Wblx .overflow--hidden__373c0__2B0kz')

                review_ratings = []

                for rating in ratings:
                    review_ratings.append(float(re.sub(' star rating', 
                                                       '',rating.attrs['aria-label'])))
                #### Dates
                dates = soup.select('.margin-b1-5__373c0__2Wblx .css-e81eai')

                review_dates = []

                for date in dates:
                    review_dates.append(date.string)

                #### Text
                texts = soup.select('.css-n6i4z7 .raw__373c0__3rcx7')
                review_texts = []

                for text in texts:
                    review_texts.append(text.get_text())

                name_business_mult = [business_name] * len(review_names)
                url_business_mult = [business_url] * len(review_names)

                review_data = list(zip(name_business_mult, url_business_mult, 
                                   review_names, review_ratings, review_dates, 
                                   review_texts))

                df_reviews = pd.DataFrame(data=review_data,
                                          columns = ['name_business', 'url', 
                                                     'username', 'rating', 
                                                     'date_review', 'text'])

                with open('memphis_reviews.csv', 'a', newline='') as f:
                        df_reviews.to_csv(f, index=False, header=False)

                print('Review page ' + str(i))
                time.sleep(4)

            except:
                print('Error in second stage')

        print(business_name)
        time.sleep(4)
                    
    except:
        print('Error in first stage')

The Second Line https://www.yelp.com/biz/the-second-line-memphis
Review page 0
Review page 1
Review page 2
Review page 3
Review page 4
The Second Line
Bounty On Broad https://www.yelp.com/biz/bounty-on-broad-memphis
Review page 0
Review page 1
Review page 2
Review page 3
Review page 4
Bounty On Broad
Curfew https://www.yelp.com/biz/curfew-memphis
Review page 0
Review page 1
Review page 2
Review page 3
Review page 4
Curfew
The Liquor Store https://www.yelp.com/biz/the-liquor-store-memphis-3
Review page 0
Review page 1
Review page 2
Review page 3
Review page 4
The Liquor Store
Beauty Shop Restaurant https://www.yelp.com/biz/beauty-shop-restaurant-memphis-3
Review page 0
Review page 1
Review page 2
Review page 3
Review page 4
Beauty Shop Restaurant


In [7]:
test = pd.read_csv('memphis_reviews.csv', header=None, encoding='ISO-8859-1')

In [8]:
test

,0,1,2,3,4,5
0,The Second Line,https://www.yelp.com/biz/the-second-line-memphis,Nikki J.,5.0,12/3/2020,This is the lunch break you deserve. First sco...
1,The Second Line,https://www.yelp.com/biz/the-second-line-memphis,Robert E.,5.0,11/3/2020,Went last Saturday night with my girlfriend. S...
2,The Second Line,https://www.yelp.com/biz/the-second-line-memphis,Diana R.,5.0,7/22/2020,"Great drinks, great food, great service every ..."
3,The Second Line,https://www.yelp.com/biz/the-second-line-memphis,Roxy M.,5.0,5/2/2021,This is a great spot to eat. They turned the m...
4,The Second Line,https://www.yelp.com/biz/the-second-line-memphis,Youngchai S.,3.0,4/8/2021,"Food is tasty, but overpriced.BBQ Shrimp: best..."
...,...,...,...,...,...,...
232,Beauty Shop Restaurant,https://www.yelp.com/biz/beauty-shop-restauran...,Kara L.,5.0,12/1/2019,"Our brunch tables do book up very quickly, esp..."
233,Beauty Shop Restaurant,https://www.yelp.com/biz/beauty-shop-restauran...,Sandra S.,5.0,8/13/2018,We just had an excellent brunch experience mad...
234,Beauty Shop Restaurant,https://www.yelp.com/biz/beauty-shop-restauran...,A O.,4.0,1/4/2021,Super delicious restaurant with such beautiful...
235,Beauty Shop Restaurant,https://www.yelp.com/biz/beauty-shop-restauran...,Xinyi Z.,2.0,3/28/2019,We went here primarily for the decor but were ...
